In [43]:
import pandas as pd
import numpy as np

In [44]:
# all_did = pd.read_csv('did資料筆數統計.csv')

In [49]:
behavior_merge = pd.read_csv('合併行為資料-20200501~20210430-4.csv')

In [50]:
behavior_merge.loc[[0,1],['did','DateTime']].values

array([['00431b99-199f-43d3-20db-aa535cf4c809',
        '2020-07-16 13:26:12.654'],
       ['00431b99-199f-43d3-20db-aa535cf4c809',
        '2020-08-26 12:06:22.179']], dtype=object)

In [51]:
behavior_merge.iloc[1,0]

9042

In [52]:
did_last = behavior_merge.loc[0].did  # 初始值
index_dict = {}  # 預計長這樣: { did1:[[traffict_index, purchase_index], [traffict_index, purchase_index]],
                 #             did2:[[traffict_index, purchase_index]}
                 # 每個did不一定只有一組對照index, 若無traffict_index便以XXX記錄

# 一行一行全部輪流跑
for i in range(len(behavior_merge)):
    did_now = behavior_merge.loc[i].did  # 每行會先檢查目前的did
     
    # 確認目前的did跟前一個相同
    if did_now == did_last:
        
        # 如果此row的行為是traffic就記錄index
        if behavior_merge.loc[i].bh == 'traffic':
            # traffic_index = behavior_merge.iloc[i,0]
            traffic_row = i
        
        # 如果此row的行為是purchase就記錄index, 並且將對照組加進index_dict
        elif behavior_merge.loc[i].bh == 'purchase':         
            # purchase_index = behavior_merge.iloc[i,0]
            purchase_row = i
            # pair_list = [traffic_index, purchase_index]  # 兩個index是一組
            pair_list = [traffic_row, purchase_row]
            if index_dict.get(did_now) == None: # 如果是此did第一次有purchase, 要先創建新list再加入
                index_dict[did_now] = []
                index_dict[did_now].append(pair_list)
            else:              
                index_dict[did_now].append(pair_list)
        
        # 如果此row的行為並非traffic或purchase, 就跳過
        else:
            pass
    
    # 若目前的did跟前一個不同，代表進入下一輪
    else:          
        did_last = did_now  # 需重新設定新一輪的did
        # traffic_index = 'XXX'  # 先將traffic_index歸零
        traffic_row = 'XXX'
        
        # 如果此row的行為是traffic就記錄index
        if behavior_merge.loc[i].bh == 'traffic':
            # traffic_index = behavior_merge.iloc[i,0]
            traffic_row = i
        
        # 如果此row的行為是purchase, 代表此did第一個行為就是purchase, 直接記錄index, 並且將對照組加進index_dict
        elif behavior_merge.loc[i].bh == 'purchase':         
            # purchase_index = behavior_merge.iloc[i,0]
            purchase_row = i
            # pair_list = [traffic_index, purchase_index]  # 兩個index是一組
            pair_list = [traffic_row, purchase_row]
            index_dict[did_now] = []
            index_dict[did_now].append(pair_list)
            
        # 如果此row的行為並非traffic或purchase, 就跳過
        else:
            pass
    
index_dict

{'00431b99-199f-43d3-20db-aa535cf4c809': [[29, 41], [29, 42]],
 '0069c666-55fe-4970-bc24-78d3abffab40': [[205, 216], [205, 217]],
 '013af2d5-d5d6-4abf-911b-92ab8d9caa63': [[313, 322], [313, 323]],
 '019c7c2d-d9ea-43bf-a31f-e50d889b0cb2': [[350, 362]],
 '02942eff-1607-4e08-adfd-aac4f9e2f54c': [['XXX', 506],
  ['XXX', 507],
  ['XXX', 541]],
 '02dd782a-7185-4bee-bcd9-a1156e73fcfb': [[860, 863]],
 '0305428a-1e0e-428c-a1ec-70e13832a513': [['XXX', 942],
  ['XXX', 943],
  ['XXX', 944],
  ['XXX', 1231],
  ['XXX', 1232],
  ['XXX', 1233]],
 '03aa3792-2235-4a86-af89-1c404118ad02': [[1416, 1421],
  [1452, 1469],
  [1472, 1477],
  [1472, 1478]],
 '03b87c79-2161-4360-0f44-9a17bf776145': [['XXX', 1530], ['XXX', 1531]],
 '03d82043-a434-4176-8aa9-5d25173f0e6a': [[1635, 1773]],
 '03ea2919-7261-4841-0de6-a50440638e6c': [[1883, 1886]],
 '05172607-51fe-4a29-66f1-87b42f3b580a': [[1968, 1970],
  [1974, 1976],
  [1981, 1987]],
 '07275020-9b79-4847-a17a-faa848b9eecd': [[2017, 2020]],
 '076e55a2-78fc-4010-b399-

In [53]:
# index_dict['12675517-09f3-42f8-bf71-d18d2409dca9'][1]

In [54]:
# time1 = behavior_merge.loc[index_dict['006053b9-9705-4916-4a94-4130a988093e'][0][0]].DateTime
# time2 = behavior_merge.loc[index_dict['006053b9-9705-4916-4a94-4130a988093e'][0][1]].DateTime
# time2

In [55]:
import datetime
did_list = np.unique(behavior_merge.did)  # 整理出所有did
deltatime_list = []  # 結果想記錄所有traffic與purchase的時間間隔
did_deltatime={}

for did in did_list:  # 輪流讀取每個did的index對照組
    temp_list=[]
    flag='no purchase'
    delta_time=0
    for i in range(len(index_dict[did])):  # 依照每個did記錄到的index對照組跑迴圈
        if index_dict[did][i][0] == 'XXX':  # 如果沒有通過traffic進行購買就先不計算
            pass
        else:
            flag='purchase'
            print(behavior_merge.loc[index_dict[did][i][0]])
            traffic_time = behavior_merge.loc[index_dict[did][i][0]].DateTime
            purchase_time = behavior_merge.loc[index_dict[did][i][1]].DateTime
            time1 = pd.to_datetime(traffic_time)
            time2 = pd.to_datetime(purchase_time)
            delta_time_now=(time2-time1).seconds
           
            if delta_time==delta_time_now: #同一筆購買不列入計算
                pass
            else:
                delta_time = delta_time_now
                temp_list.append(delta_time)
        
    
    if flag=='purchase':
        did_deltatime[did]=temp_list

index                                                  14710
dc                                                       web
did                     00431b99-199f-43d3-20db-aa535cf4c809
uid         g7sLZR5omJkB3pOuY0DG1NlU%2FCV7uP7cfPmeGs4Q4xU%3D
DateTime                             2020-08-30 14:44:44.273
bh                                                   traffic
cs                                       tw.search.yahoo.com
cm                                                  referral
cn                                                       NaN
pr                                                       NaN
pr1id                                                    NaN
pr1pr                                                    NaN
pr1qt                                                    NaN
cu                                                       NaN
tr                                                       NaN
ti                                                       NaN
Name: 29, dtype: object


Name: 6951, dtype: object
index                                                    9174
dc                                                        app
did                      1e946fb9-f236-48ce-ae6c-0b8dfbdcaf9e
uid         tqDywie%2BIzUx5tSXL7PpKw55fW5M%2FIgcOweP1N4ck5...
DateTime                              2020-08-07 04:25:28.743
bh                                                    traffic
cs                                                       mweb
cm                                            download_banner
cn                                              ordercomplete
pr                                                        NaN
pr1id                                                     NaN
pr1pr                                                     NaN
pr1qt                                                     NaN
cu                                                        NaN
tr                                                        NaN
ti                                          

Name: 26999, dtype: object
index                                                  13466
dc                                                       web
did                     38259310-a191-4fba-a7c7-411ff54de910
uid         zDtpDUVzv4mfiv0rnjHqcFVrwgJTjPv%2BXIfTAXrZ9WU%3D
DateTime                             2020-09-07 07:19:44.208
bh                                                   traffic
cs                                                      FBAD
cm                                       202008142200B348828
cn                                                       NaN
pr                                                       NaN
pr1id                                                    NaN
pr1pr                                                    NaN
pr1qt                                                    NaN
cu                                                       NaN
tr                                                       NaN
ti                                                       N

Name: 40890, dtype: object
index                                                 8606
dc                                                     web
did                   5c9e993c-879d-45dc-fb6d-cdefda8f47f4
uid         r4HcF3ui1qlRTK4vpx5EaZ8dPWI1ghUnshf2PU6Y60g%3D
DateTime                           2020-08-21 12:29:09.443
bh                                                 traffic
cs                                               affiliate
cm                                                 koodata
cn                                                     NaN
pr                                                     NaN
pr1id                                                  NaN
pr1pr                                                  NaN
pr1qt                                                  NaN
cu                                                     NaN
tr                                                     NaN
ti                                                     NaN
Name: 40890, dtype: object
in

Name: 61182, dtype: object
index                                       9903
dc                                           web
did         8df8eb2c-ec62-4a1f-e656-ceea7f1914e8
uid                                          NaN
DateTime                 2020-08-23 05:48:25.619
bh                                       traffic
cs                             preview.page.link
cm                                      referral
cn                                           NaN
pr                                           NaN
pr1id                                        NaN
pr1pr                                        NaN
pr1qt                                        NaN
cu                                           NaN
tr                                           NaN
ti                                           NaN
Name: 61251, dtype: object
index                                                  13287
dc                                                       web
did                     8f81e24b-96db-4f

index                                       5083
dc                                           web
did         b86d47d9-abb5-47ab-ba03-e79116a51454
uid                                          NaN
DateTime                 2020-08-24 04:37:08.773
bh                                       traffic
cs                                        google
cm                                           cpc
cn                                   acquisition
pr                                           NaN
pr1id                                        NaN
pr1pr                                        NaN
pr1qt                                        NaN
cu                                           NaN
tr                                           NaN
ti                                           NaN
Name: 85863, dtype: object
index                                       4140
dc                                           web
did         b86d664b-3f43-428b-8e2a-3770b579b2ed
uid                                       

Name: 95617, dtype: object
index                                       8679
dc                                           web
did         cdd5096d-3c96-458e-461f-0f710a6a127c
uid                                          NaN
DateTime                 2020-08-05 05:06:23.833
bh                                       traffic
cs                              www.facebook.com
cm                                      referral
cn                                           NaN
pr                                           NaN
pr1id                                        NaN
pr1pr                                        NaN
pr1qt                                        NaN
cu                                           NaN
tr                                           NaN
ti                                           NaN
Name: 95679, dtype: object
index                                                   13772
dc                                                        web
did                      ce7c7a62-b77e

Name: 99678, dtype: object
index                                                   19212
dc                                                        web
did                      dbd6f038-dc53-4b82-ffc0-81dbae5982e3
uid         BWUMM8W6eYKWz%2FWpVveGhymW16otb81awDW7bXB%2Fc8...
DateTime                              2020-11-05 01:18:05.224
bh                                                    traffic
cs                                             l.facebook.com
cm                                                   referral
cn                                                        NaN
pr                                                        NaN
pr1id                                                     NaN
pr1pr                                                     NaN
pr1qt                                                     NaN
cu                                                        NaN
tr                                                        NaN
ti                                         

Name: 109849, dtype: object
index                                       3762
dc                                           web
did         ffd2c84e-34ec-4b96-b7f0-92c7df860081
uid                                          NaN
DateTime                 2020-08-05 04:04:57.610
bh                                       traffic
cs                                        google
cm                                           cpc
cn                                   acquisition
pr                                           NaN
pr1id                                        NaN
pr1pr                                        NaN
pr1qt                                        NaN
cu                                           NaN
tr                                           NaN
ti                                           NaN
Name: 109849, dtype: object
index                                       3762
dc                                           web
did         ffd2c84e-34ec-4b96-b7f0-92c7df860081
uid          

In [56]:
#輸出檔案

output_list = []
a = 0
for did in did_deltatime.keys():
    output_list.append({})
    output_list[a]['did'] = did
    
    for num in range(len(did_deltatime[did])):
        output_list[a][num] = did_deltatime[did][num]
    a += 1
    #output_list.append({
     #   'did': did,
      #  num: did_deltatime[did][num]
    #})

output = pd.DataFrame(output_list)
print(len(output['did'].unique()))
output.to_csv('diff_4.csv', index=False)

366


In [23]:
###舊的不要跑
import datetime
did_list = np.unique(behavior_merge.did)  # 整理出所有did
deltatime_list = []  # 結果想記錄所有traffic與purchase的時間間隔

for did in did_list:  # 輪流讀取每個did的index對照組
    for i in range(len(index_dict[did])):  # 依照每個did記錄到的index對照組跑迴圈
        if index_dict[did][i][0] == 'XXX':  # 如果沒有通過traffic進行購買就先不計算
            pass
        else:
            traffic_time = behavior_merge.loc[index_dict[did][i][0]].DateTime
            purchase_time = behavior_merge.loc[index_dict[did][i][1]].DateTime
            time1 = pd.to_datetime(traffic_time)
            time2 = pd.to_datetime(purchase_time)
            delta_time = time2-time1
            delta_time = delta_time.seconds
            deltatime_list.append(delta_time)
            print(traffic_time,purchase_time,delta_time)

2021-04-21 23:45:22.590 2021-04-22 00:25:53.026 2430
2021-04-20 09:19:00.315 2021-04-20 09:35:47.675 1007
2021-04-28 23:50:59.207 2021-04-29 00:09:03.103 1083
2021-04-28 23:50:59.207 2021-04-29 00:09:03.103 1083
2021-04-28 23:50:59.207 2021-04-29 00:09:03.103 1083
2021-04-17 15:03:22.043 2021-04-19 04:30:29.563 48427
2021-04-17 15:03:22.043 2021-04-19 04:30:29.563 48427
2021-04-19 02:24:56.772 2021-04-19 02:27:46.734 169
2021-04-29 14:08:09.021 2021-04-29 14:13:16.974 307
2021-04-29 09:31:55.832 2021-04-29 09:37:26.408 330
2021-04-30 02:27:19.880 2021-04-30 02:40:31.671 791
2021-04-30 02:42:37.293 2021-04-30 02:50:26.698 469
2021-04-30 02:42:37.293 2021-04-30 02:50:26.698 469
2021-04-30 02:42:37.293 2021-04-30 02:50:26.698 469
2021-04-30 02:42:37.293 2021-04-30 02:50:26.698 469
2021-04-26 16:18:43.253 2021-04-26 16:26:58.989 495
2021-04-26 16:18:43.253 2021-04-26 16:26:58.989 495
2021-04-27 02:26:22.494 2021-04-27 02:29:36.974 194
2021-04-27 02:26:22.494 2021-04-27 02:29:36.974 194
202

In [24]:
###舊的不要跑
import datetime
did_list = np.unique(behavior_merge.did)  # 整理出所有did
deltatime_list = []  # 結果想記錄所有traffic與purchase的時間間隔
did_deltatime={}

for did in did_list:  # 輪流讀取每個did的index對照組
    temp_list=[]
    flag='no purchase'
    for i in range(len(index_dict[did])):  # 依照每個did記錄到的index對照組跑迴圈
        if index_dict[did][i][0] == 'XXX':  # 如果沒有通過traffic進行購買就先不計算
            pass
        else:
            flag='purchase'
            traffic_time = behavior_merge.loc[index_dict[did][i][0]].DateTime
            purchase_time = behavior_merge.loc[index_dict[did][i][1]].DateTime
            time1 = pd.to_datetime(traffic_time)
            time2 = pd.to_datetime(purchase_time)
            delta_time = time2-time1
            delta_time = delta_time.seconds
            temp_list.append(delta_time)
    if flag=='purchase':
        did_deltatime[did]=temp_list